#### Load the data

In [5]:
import pandas as pd
import pickle
import math
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [6]:
data_dir = "../../data/"
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(data_dir + name + '.pkl', 'rb') as f:
        return pickle.load(f)
# load data
Old_Labeled_data = load_obj("SBdata")
BasicFeatures = load_obj("pre_features-v2")
pre_features = BasicFeatures

def transform_data(Old_Labeled_data):
    labeled_edges = []
    for Old_Labeled_edge in Old_Labeled_data:
        label = int(Old_Labeled_edge[1])
        source = int(Old_Labeled_edge[0][0])
        sink = int(Old_Labeled_edge[0][1])
        labeled_edges.append((source, sink, label))
    return labeled_edges
labeled_edges = transform_data(Old_Labeled_data)

In [137]:
test_edges = labeled_edges[0:20]

In [7]:
# Function
#Salton Similarity
def salton_similarity(node1, node2):
    n1 = pre_features[node1]
    n2 = pre_features[node2]
    common_neighors = list(set(n1[2]).intersection(n2[2]))
    inter = len(common_neighors)
    degree_out_flow = n1[6]
    degree_in_flow = n2[4]
    
    if inter == 0:
        return 0
    else:
        try:
            sqrt_of_degree = math.sqrt(degree_out_flow * degree_in_flow)
            salton = inter / sqrt_of_degree
            probability = 1 /(1 - math.log(salton)*0.2)
            return probability
        except:
            return 0

#Cosine
def Cosine(Node1, Node2):
    n1 = pre_features[Node1]
    n2 = pre_features[Node2]
    common_neighors = list(set(n1[2]).intersection(n2[2]))
    lm = len(common_neighors)
    if lm == 0:
        return 0
    else:
        return (0.0+lm)/(len(n1[2])*len(n2[2]))

def get_jaccard_coefficient(source, sink):
    """
    in: source::Node object
    in: sink::Node object
    return: jaccard's cofficient::numeric
    """
    # transform
    neighbours_of_source_list = BasicFeatures[source][2]
    neighbours_of_sink_list = BasicFeatures[sink][2]
    
    neigbours_set_of_source = set(neighbours_of_source_list)
    neigbours_set_of_sink = set(neighbours_of_sink_list)
    union_neighbours = neigbours_set_of_source | neigbours_set_of_sink
    common_neighbours = neigbours_set_of_source & neigbours_set_of_sink
    if len(union_neighbours)==0:
        return 0.0
    return(len(common_neighbours)/len(union_neighbours))

def get_preferential_attachment(source, sink):
    # transform
    neighbours_of_source_list = BasicFeatures[source][2]
    neighbours_of_sink_list = BasicFeatures[sink][2]
    
    neigbours_set_of_source = set(neighbours_of_source_list)
    neigbours_set_of_sink = set(neighbours_of_sink_list)
    
    return len(neigbours_set_of_source)*len(neigbours_set_of_sink)

def get_adamic_adar(source, sink):
    # transform
    neighbours_of_source_list = BasicFeatures[source][2]
    neighbours_of_sink_list = BasicFeatures[sink][2]

    neigbours_set_of_source = set(neighbours_of_source_list)
    neigbours_set_of_sink = set(neighbours_of_sink_list)
    common_neighbours = neigbours_set_of_source & neigbours_set_of_sink
    # get the summation
    score = 0
    for common_node in common_neighbours:
        if math.log(len(BasicFeatures[common_node][2])) == 0:
            return 0.0
        score = score + 1/math.log(len(BasicFeatures[common_node][2]))
    return score

def get_resource_allocation(source, sink):
    neighbours_of_source_list = BasicFeatures[source][2]
    neighbours_of_sink_list = BasicFeatures[sink][2]
#     print(neighbours_of_source_list)
#     print(neighbours_of_sink_list)
    neigbours_set_of_source = set(neighbours_of_source_list)
    neigbours_set_of_sink = set(neighbours_of_sink_list)
    
    common_neighbours = neigbours_set_of_source & neigbours_set_of_sink
#     print(common_neighbours)
    score=0
    for common_node in common_neighbours:
        # number of the neighbours of the common_node
        try:
            single_common_node_score = 1/BasicFeatures[common_node][0]
        except:
            single_common_node_score=0
        score = score + single_common_node_score
    return score
    

# def get_resource_allocation_v2(source, sink):
#     in_neighbours_of_source_list = set(BasicFeatures[source][3])
#     in_neighbours_of_sink_list = set(BasicFeatures[sink][3])

#     out_neighbours_of_source_list = set(BasicFeatures[source][5])
#     out_neighbours_of_sink_list = set(BasicFeatures[sink][5])

#     common_neighbours_1 = out_neighbours_of_source_list & in_neighbours_of_sink_list
#     common_neighbours_2 = in_neighbours_of_source_list & out_neighbours_of_sink_list

#     score_x=0
#     score_y=0
#     for common_node in common_neighbours_1:
#         # number of the neighbours of the common_node
#         try:
#             # only count the out neighobour
#             single_common_node_score = 1/BasicFeatures[common_node][5]
#         except:
#             single_common_node_score=0
#         score_x = score_x + single_common_node_score
    
#     for common_node in common_neighbours_2:
#         # number of the neighbours of the common_node
#         try:
#             # only count the out neighobour
#             single_common_node_score = 1/BasicFeatures[common_node][5]
#         except:
#             single_common_node_score=0
#         score_y = score_y + single_common_node_score
#     score = score_x+score_y
#     return score
    
    
    
    
# how similar are the outbound neighbors of source to sink
# either JA, PA, AA
def get_outbound_similarity_score(source, sink, metric):
    # get the outbound_node of source
    outbound_node_for_source_set = set(BasicFeatures[source][5])
    summation = 0
    for outbound_node_for_source in outbound_node_for_source_set:
        summation =summation + metric(sink,outbound_node_for_source)
    if len(outbound_node_for_source_set) == 0:
        return 0
    score = 1/len(outbound_node_for_source_set)*summation
    return score

# either JA, PA, AA
def get_inbound_similarity_score(source, sink, metric):
    # get the inbound_node of sink
    inbound_node_for_sink_set = set(BasicFeatures[sink][3])
    summation = 0
    for inbound_node_for_sink in inbound_node_for_sink_set:
        summation =summation + metric(source,inbound_node_for_sink)
    if len(inbound_node_for_sink_set) == 0:
        return 0
    score = 1/len(inbound_node_for_sink_set)*summation
    return score

def get_common_neighbours(node1, node2):
    try:
        n1 = pre_features[node1]
        n2 = pre_features[node2]
        common_neighors = list(set(n1[2]).intersection(n2[2]))
        return common_neighors
    except:
        return 0

def get_training_df(final_edges):
    training_df = pd.DataFrame()
    for edge in tqdm(final_edges, mininterval=600):
        source = edge[0]
        sink = edge[1]
        label = edge[2]
        common_neighbours = get_common_neighbours(source,sink)
        num_of_neighbours_source=BasicFeatures[source][0]
        num_of_in_neighbours_source=BasicFeatures[source][4]
        num_of_out_neighbours_source=BasicFeatures[source][6]

        num_of_neighbours_sink=BasicFeatures[sink][0]
        num_of_in_neighbours_sink=BasicFeatures[sink][4]
        num_of_out_neighbours_sink=BasicFeatures[sink][6]
        
        num_of_neighbours_sum=BasicFeatures[source][0] + BasicFeatures[sink][0]
        num_of_in_neighbours_sum=BasicFeatures[source][4] + BasicFeatures[sink][4]
        num_of_out_neighbours_sum=BasicFeatures[source][6] + BasicFeatures[sink][6]
        
        salton_similarity_score = salton_similarity(source, sink)
        cosine = Cosine(source, sink)
        jaccard_coefficient = get_jaccard_coefficient(source, sink)
        preferential_attachment = get_preferential_attachment(source, sink)
        adamic_adar = get_adamic_adar(source, sink)
        resource_allocation = get_resource_allocation(source, sink)

#         salton_similarity_score_out = get_outbound_similarity_score(source, sink, salton_similarity)
#         cosine_out = get_outbound_similarity_score(source, sink, Cosine)
#         jaccard_coefficient_out = get_outbound_similarity_score(source, sink, get_jaccard_coefficient)
#         preferential_attachment_out = get_outbound_similarity_score(source, sink, get_preferential_attachment)
#         adamic_adar_out = get_outbound_similarity_score(source, sink, get_adamic_adar)
#         resource_allocation_out = get_outbound_similarity_score(source, sink, get_resource_allocation)

#         salton_similarity_score_in = get_inbound_similarity_score(source, sink, salton_similarity)
#         cosine_in = get_inbound_similarity_score(source, sink, Cosine)
#         jaccard_coefficient_in = get_inbound_similarity_score(source, sink, get_jaccard_coefficient)
#         preferential_attachment_in = get_inbound_similarity_score(source, sink, get_preferential_attachment)
#         adamic_adar_in = get_inbound_similarity_score(source, sink, get_adamic_adar)
#         resource_allocation_in = get_inbound_similarity_score(source, sink, get_resource_allocation)

# add the basic features
        df_row = pd.DataFrame([
                               source, 
                               sink, 
                               label,
                               num_of_neighbours_source,
                               num_of_in_neighbours_source,
                               num_of_out_neighbours_source,
                               num_of_neighbours_sink,
                               num_of_in_neighbours_sink,
                               num_of_out_neighbours_sink,
                               num_of_neighbours_sum,
                               num_of_in_neighbours_sum,
                               num_of_out_neighbours_sum,      
                               salton_similarity_score, 
                               cosine, 
                               jaccard_coefficient,
                               preferential_attachment, 
                               adamic_adar, 
                               resource_allocation
#                                salton_similarity_score_out,
#                                cosine_out,
#                                jaccard_coefficient_out,
#                                preferential_attachment_out,
#                                adamic_adar_out,
#                                resource_allocation_out,
#                                salton_similarity_score_in,
#                                cosine_in,
#                                jaccard_coefficient_in,
#                                preferential_attachment_in,
#                                adamic_adar_in,
#                                resource_allocation_in
                              ]).T
        training_df = training_df.append(df_row)
    training_df.rename(columns={
        0:'source', 
        1:'sink', 
        2:'label',
        3:'num_of_neighbours_source',
        4:'num_of_in_neighbours_source',
        5:'num_of_out_neighbours_source',
        6:'num_of_neighbours_sink',
        7:'num_of_in_neighbours_sink',
        8:'num_of_out_neighbours_sink',
        9:'num_of_neighbours_sum',
        10:'num_of_in_neighbours_sum',
        11:'num_of_out_neighbours_sum',      
        12:'salton_similarity_score', 
        13:'cosine', 
        14:'jaccard_coefficient',
        15:'preferential_attachment', 
        16:'adamic_adar', 
        17:'resource_allocation'
#         19:'salton_similarity_score_out',
#         20:'cosine_out',
#         21:'jaccard_coefficient_out',
#         22:'preferential_attachment_out',
#         23:'adamic_adar_out',
#         24:'resource_allocation_out',
#         25:'salton_similarity_score_in',
#         26:'cosine_in',
#         27:'jaccard_coefficient_in',
#         28:'preferential_attachment_in',
#         29:'adamic_adar_in',
#         30:'resource_allocation_in'           
    },inplace=True)
    training_df[['source', 'sink', 'label']] = training_df[['source', 'sink', 'label']].astype(int)
    return training_df

# data 需要为array
def rescale_min_max(data): 
    """
    min-max normalisation
    """
    scaler = MinMaxScaler()
    scaler.fit(data)
    result = scaler.transform(data)
    return pd.DataFrame(result)

def standardise(data):
    """remove the mean and transform to unit variance"""
    scaler = StandardScaler()
    scaler.fit(data)
    result = scaler.transform(data)
    return pd.DataFrame(result)


In [ ]:
# normalise the training data
training_df = get_training_df(labeled_edges)
final_labels_df = training_df.iloc[:,2]
measurement_to_normal = training_df.iloc[:,3:30]
# 使用标准化
final_training_data_df = standardise(measurement_to_normal)
final_training_data_df.rename(columns={
        0:'num_of_neighbours_source',
        1:'num_of_in_neighbours_source',
        2:'num_of_out_neighbours_source',
        3:'num_of_neighbours_sink',
        4:'num_of_in_neighbours_sink',
        5:'num_of_out_neighbours_sink',
        6:'num_of_neighbours_sum',
        7:'num_of_in_neighbours_sum',
        8:'num_of_out_neighbours_sum',      
        9:'salton_similarity_score', 
        10:'cosine', 
        11:'jaccard_coefficient',
        12:'preferential_attachment', 
        13:'adamic_adar', 
        14:'resource_allocation'
#         15:'salton_similarity_score_out',
#         16:'cosine_out',
#         17:'jaccard_coefficient_out',
#         18:'preferential_attachment_out',
#         19:'adamic_adar_out',
#         20:'resource_allocation_out'
#         21:'salton_similarity_score_in',
#         22:'cosine_in',
#         23:'jaccard_coefficient_in',
#         24:'preferential_attachment_in',
#         25:'adamic_adar_in',
#         26:'resource_allocation_in'           
    },inplace=True)


In [18]:
save_obj(training_df,'un_normalised_final_training_data_df_rf')

In [29]:
training_df

,source,sink,label,num_of_neighbours_source,num_of_in_neighbours_source,num_of_out_neighbours_source,num_of_neighbours_sink,num_of_in_neighbours_sink,num_of_out_neighbours_sink,num_of_neighbours_sum,num_of_in_neighbours_sum,num_of_out_neighbours_sum,salton_similarity_score,cosine,jaccard_coefficient,preferential_attachment,adamic_adar,resource_allocation
0,4066935,3642884,1,285.0,142.0,143.0,635.0,269.0,366.0,920.0,411.0,509.0,0.737216,1.823456e-04,0.054010,90915.0,5.088169,0.100367
0,4066935,4394015,1,285.0,142.0,143.0,329.0,107.0,222.0,614.0,249.0,365.0,0.779224,3.199488e-04,0.074257,47025.0,4.255364,0.036850
0,4066935,2795881,1,285.0,142.0,143.0,925.0,568.0,357.0,1210.0,710.0,500.0,0.754474,2.124229e-04,0.067389,141702.0,8.210339,0.090748
0,4066935,3314478,1,285.0,142.0,143.0,11587.0,1231.0,10356.0,11872.0,1373.0,10499.0,0.755244,2.513404e-05,0.007836,2187394.0,11.987406,0.147143
0,4066935,1084494,1,285.0,142.0,143.0,359.0,221.0,138.0,644.0,363.0,281.0,0.802664,5.082344e-04,0.127139,52668.0,10.020895,0.414566
0,4066935,3586277,1,285.0,142.0,143.0,80.0,33.0,47.0,365.0,175.0,190.0,0.788730,7.894737e-04,0.074380,10659.0,3.326567,0.110527
0,4066935,378745,1,285.0,142.0,143.0,706.0,480.0,226.0,991.0,622.0,369.0,0.729422,2.037672e-04,0.059942,107844.0,6.075542,0.076190
0,4066935,3670317,1,285.0,142.0,143.0,6626.0,777.0,5849.0,6911.0,919.0,5992.0,0.697199,2.012275e-05,0.006299,1225158.0,5.045627,0.031914
0,4066935,1588917,1,285.0,142.0,143.0,579.0,65.0,514.0,864.0,207.0,657.0,0.766390,1.272612e-04,0.029207,110979.0,2.683657,0.013350
0,4066935,1337901,1,285.0,142.0,143.0,228.0,131.0,97.0,513.0,273.0,240.0,0.585979,6.155740e-05,0.011461,30096.0,0.521257,0.002167


In [ ]:
final_labels_df = training_df.iloc[:,2]

In [12]:
save_obj(final_labels_df,'final_labels_df-v1')

In [28]:
final_training_data_df

,num_of_neighbours_source,num_of_in_neighbours_source,num_of_out_neighbours_source,num_of_neighbours_sink,num_of_in_neighbours_sink,num_of_out_neighbours_sink,num_of_neighbours_sum,num_of_in_neighbours_sum,num_of_out_neighbours_sum,salton_similarity_score,cosine,jaccard_coefficient,preferential_attachment,adamic_adar,resource_allocation
0,-0.101935,0.253971,-0.107772,-0.033526,0.484222,-0.041148,-0.072108,0.546425,-0.081390,1.259111,-0.172340,1.182078,-0.040877,0.398795,0.073546
1,-0.101935,0.253971,-0.107772,-0.046859,0.018131,-0.047467,-0.084322,0.139991,-0.087180,1.385165,-0.121004,1.762535,-0.044940,0.307209,-0.039631
2,-0.101935,0.253971,-0.107772,-0.020889,1.344475,-0.041543,-0.060532,1.296570,-0.081752,1.310898,-0.161119,1.565622,-0.036175,0.742148,0.056407
3,-0.101935,0.253971,-0.107772,0.443697,3.251995,0.397231,0.365041,2.959937,0.320267,1.313209,-0.230991,-0.141624,0.153198,1.157522,0.156895
4,-0.101935,0.253971,-0.107772,-0.045552,0.346121,-0.051153,-0.083124,0.426000,-0.090557,1.455501,-0.050761,3.278548,-0.044417,0.941259,0.633403
5,-0.101935,0.253971,-0.107772,-0.057709,-0.194775,-0.055146,-0.094260,-0.045664,-0.094216,1.413690,0.054161,1.766053,-0.048306,0.205067,0.091651
6,-0.101935,0.253971,-0.107772,-0.030432,1.091290,-0.047291,-0.069274,1.075792,-0.087019,1.235722,-0.164348,1.352128,-0.039309,0.507379,0.030467
7,-0.101935,0.253971,-0.107772,0.227526,1.945790,0.199456,0.167023,1.820920,0.139059,1.139029,-0.232860,-0.185699,0.064122,0.394117,-0.048427
8,-0.101935,0.253971,-0.107772,-0.035966,-0.102707,-0.034653,-0.074343,0.034619,-0.075439,1.346654,-0.192890,0.471041,-0.039019,0.134365,-0.081506
9,-0.101935,0.253971,-0.107772,-0.051260,0.087181,-0.052952,-0.088353,0.200203,-0.092205,0.805288,-0.217402,-0.037697,-0.046507,-0.103440,-0.101432


In [8]:
training_data_v1 = load_obj("un_normalised_final_training_data_df_rf")

final_training_data_df = training_data_v1.iloc[:,3:30]
final_labels_df = training_data_v1.iloc[:,2]


In [ ]:
X=final_training_data_df
# count=0
# get the data and label
y=final_labels_df

# training model
from sklearn.model_selection import train_test_split
X_t, X_test, y_t, y_test = train_test_split(X,y)
X_train, X_validation, y_train, y_validation  = train_test_split(X_t,y_t)

# Gridsearch settings
rf = RandomForestClassifier()
X_train = X_t
y_train = y_t

pipeline = Pipeline([
       ('clf', RandomForestClassifier(criterion='entropy'))
   ])
parameters = {
       'clf__n_estimators': (50, 300, 500),
       'clf__max_depth': (10, 100),
       'clf__min_samples_split': (5, 10, 100),
       'clf__min_samples_leaf': (5, 10, 100)
}
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1,
   verbose=1, scoring='roc_auc', cv=3)

grid_search.fit(X_train, y_train)
print('Best score: %0.3f' % grid_search.best_score_)
print('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))
predictions = grid_search.predict(X_test)
print('Accuracy:', accuracy_score(y_test, predictions))
print('Precision:', precision_score(y_test, predictions))
print('Recall:', recall_score(y_test, predictions))




# make the prediction

with open(data_dir + "test-public.txt", "r") as f:
     test_data = f.readlines()
test_data = [i.split() for i in test_data[1:]]
def predict():
    """
    make the prediction using the jaccard's coefficient
    """
    result = []
    for line in tqdm(test_data, mininterval=50):
        # converse to integer
        source = int(line[1].strip())
        sink = int(line[2].strip())
        common_neighbours = get_common_neighbours(source,sink)
        num_of_neighbours_source=BasicFeatures[source][0]
        num_of_in_neighbours_source=BasicFeatures[source][4]
        num_of_out_neighbours_source=BasicFeatures[source][6]

        num_of_neighbours_sink=BasicFeatures[sink][0]
        num_of_in_neighbours_sink=BasicFeatures[sink][4]
        num_of_out_neighbours_sink=BasicFeatures[sink][6]
        
        num_of_neighbours_sum=BasicFeatures[source][0] + BasicFeatures[sink][0]
        num_of_in_neighbours_sum=BasicFeatures[source][4] + BasicFeatures[sink][4]
        num_of_out_neighbours_sum=BasicFeatures[source][6] + BasicFeatures[sink][6]
        
        salton_similarity_score = salton_similarity(source, sink)
        cosine = Cosine(source, sink)
        jaccard_coefficient = get_jaccard_coefficient(source, sink)
        preferential_attachment = get_preferential_attachment(source, sink)
        adamic_adar = get_adamic_adar(source, sink)
        resource_allocation = get_resource_allocation(source, sink)

#         salton_similarity_score_out = get_outbound_similarity_score(source, sink, salton_similarity)
#         cosine_out = get_outbound_similarity_score(source, sink, Cosine)
#         jaccard_coefficient_out = get_outbound_similarity_score(source, sink, get_jaccard_coefficient)
#         preferential_attachment_out = get_outbound_similarity_score(source, sink, get_preferential_attachment)
#         adamic_adar_out = get_outbound_similarity_score(source, sink, get_adamic_adar)
#         resource_allocation_out = get_outbound_similarity_score(source, sink, get_resource_allocation)

#         salton_similarity_score_in = get_inbound_similarity_score(source, sink, salton_similarity)
#         cosine_in = get_inbound_similarity_score(source, sink, Cosine)
#         jaccard_coefficient_in = get_inbound_similarity_score(source, sink, get_jaccard_coefficient)
#         preferential_attachment_in = get_inbound_similarity_score(source, sink, get_preferential_attachment)
#         adamic_adar_in = get_inbound_similarity_score(source, sink, get_adamic_adar)
#         resource_allocation_in = get_inbound_similarity_score(source, sink, get_resource_allocation)

#         df_row = pd.DataFrame([cosine, jaccard_coefficient, preferential_attachment, adamic_adar, resource_allocation]).T
        X_test = pd.DataFrame([
                               num_of_neighbours_source,
                               num_of_in_neighbours_source,
                               num_of_out_neighbours_source,
                               num_of_neighbours_sink,
                               num_of_in_neighbours_sink,
                               num_of_out_neighbours_sink,
                               num_of_neighbours_sum,
                               num_of_in_neighbours_sum,
                               num_of_out_neighbours_sum,      
                               salton_similarity_score, 
                               cosine, 
                               jaccard_coefficient,
                               preferential_attachment, 
                               adamic_adar, 
                               resource_allocation
#                                salton_similarity_score_out,
#                                cosine_out,
#                                jaccard_coefficient_out,
#                                preferential_attachment_out,
#                                adamic_adar_out,
#                                resource_allocation_out,
#                                salton_similarity_score_in,
#                                cosine_in,
#                                jaccard_coefficient_in,
#                                preferential_attachment_in,
#                                adamic_adar_in,
#                                resource_allocation_in
                              ]).T
        single_result = grid_search.predict(X_test)[0]
        print(single_result)
        result.append((line[0], single_result))
    return result
result = predict()




# save the result

import csv
import time
'''
Description: get time
Input: 
Output: time
''' 
def nowtime():
    return time.strftime("%Y%m%d-%H%M", time.localtime())


"""
Description: Save prediction result to files
Input: (1) result
       (2) filename
Output: 
"""
def save_prediction_to_csv(result,filename):
    headers = ['id','Prediction']

    with open(filename + str(nowtime()) + ".csv", 'w', encoding = 'utf8') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        f_csv.writerows(result)
save_prediction_to_csv(result, "shawn_rf_non_normalised_")

Fitting 3 folds for each of 54 candidates, totalling 162 fits


# Save the result of the prediction

# Modelling

# Please save the training set as the csv file.

# Prediction

In [1]:
result

NameError: name 'result' is not defined